<a href="https://colab.research.google.com/github/junkgear/DrQA/blob/master/GitHub_Issue_Types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

pd.set_option('max_colwidth', 1000)

df = pd.concat([pd.read_csv(f'https://storage.googleapis.com/codenet/issue_labels/00000000000{i}.csv.gz')
                for i in range(10)])

In [2]:
df.head()

,url,repo,title,body,num_labels,labels,c_bug,c_feature,c_question,class_int
0,"""https://github.com/F5Networks/f5-openstack-lbaasv2-driver/issues/835""",F5Networks/f5-openstack-lbaasv2-driver,test_l7policies_and_rules.py:testl7basicupdate.test_policy_deployment_operand_match fails in setup error was encountered instead of active/onine,"title: test_l7policies_and_rules.py:testl7basicupdate.test_policy_deployment_operand_match fails in setup error was encountered instead of active/onine\r attachments: \r details: \r \r suggested issue type\r test bug\r \r traceback\r \r 0 traceback most recent call last :\r 1 file \ /home-local/jenkins/workspace/openstack/driver/newton/11.5.4-undercloud-vxlan/f5lbaasdriver/test/tempest/tests/scenario/test_l7policies_and_rules.py\ , line 305, in setup\r 2 super testl7basicupdate, self .setup \r 3 file \ /home-local/jenkins/workspace/openstack/driver/newton/11.5.4-undercloud-vxlan/f5lbaasdriver/test/tempest/tests/scenario/test_l7policies_and_rules.py\ , line 26, in setup\r 4 super testl7basic, self .setup \r 5 file \ /home-local/jenkins/workspace/openstack/driver/newton/11.5.4-undercloud-vxlan/f5lbaasdriver/test/tempest/tests/scenario/f5_base.py\ , line 33, in setup\r 6 self._create_load_balancer \r 7 file \ /home-local/jenki...",3,"[""P3"", ""S3"", ""test-bug""]",True,False,False,0
1,"""https://github.com/aspnet/Mvc/issues/6339""",aspnet/Mvc,testing all controllers dependency injection,i'm writing integration tests for my application and i would like to know how to test if i correctly registered all of my dependencies on startup. how can i instantiate all controllers using a testserver and look for exceptions?,1,"[""question""]",False,False,True,2
2,"""https://github.com/ionic-team/ionic-cli/issues/3044""",ionic-team/ionic-cli,testing ionic4 - serve shows two displays,\r description: \r ionic serve shows two displays\r \r steps to reproduce: \r \r c:\\ae\\adaept.com\\aeion4>ionic serve --lab --browser chrome\r > ng serve --host=0.0.0.0 --port=8100 --progress=false\r > ionic-lab http://localhost:8100 --host localhost --port 8200 --app-name aeion4 --app-version 0.0.1\r \r ok development server running!\r \r lab: http://localhost:8200\r local: http://localhost:8100\r external: http://192.168.1.67:8100\r devapp: aeion4@8100 on merry-christmas\r \r info browser window opened to http://localhost:8200!\r \r ng webpack: wait until bundle finished: /?ionicplatform=ios&ionicstatusbarpadding=true\r ng webpack: wait until bundle finished: /?ionicplatform=android\r ng date: 2018-03-28t04:53:39.582z\r ng hash: beda4693df6225dea695\r ng time: 13053ms\r ng chunk {inline} inline.bundle.js inline 3.85 kb entry rendered \r ng chunk {main} main.bundle.js main 23.5 kb initial rendered \r ng chunk {polyfills}...,1,"[""bug""]",True,False,False,0
3,"""https://github.com/thefarwind/chip-8/issues/21""",thefarwind/chip-8,tests are all broken,"when switching chip8 such that the audio, display, and input are all fed to a new function, all the tests were broken. this is blocked until after audio, display, and input are implemented as traits see 20 .",2,"[""bug"", ""tests""]",True,False,False,0
4,"""https://github.com/n-sokolov/CoffeeShop/issues/1""",n-sokolov/CoffeeShop,tests for paging,"_ context _: paging mechanism must be tested.\r \r _ problem _: tests use the same database as the main application, so there is no guarantee that tests will be run against the same environment over a collection.\r \r _ solution _: separate main database and test database; drop collection every time a test is run.",1,"[""enhancement""]",False,True,False,1


In [0]:
#split data into train/test
traindf, testdf = train_test_split(df, test_size=.15)

In [0]:
traindf.to_pickle('traindf.pkl')
testdf.to_pickle('testdf.pkl')

In [5]:
#print out stats about shape of data
print(f'Train: {traindf.shape[0]:,} rows {traindf.shape[1]:,} columns')
print(f'Test: {testdf.shape[0]:,} rows {testdf.shape[1]:,} columns')

Train: 2,698,578 rows 10 columns
Test: 476,220 rows 10 columns


In [0]:
from ktext.preprocess import processor
import dill as dpickle
import numpy as np

In [0]:
%%time

train_body_raw = traindf.body.tolist()
train_title_raw = traindf.title.tolist()

# Clean, tokenize, and apply padding / truncating such that each document length = 75th percentile for the dataset.
#  also, retain only the top keep_n words in the vocabulary and set the remaining words
#  to 1 which will become common index for rare words 

# process the issue body data
body_pp = processor(heuristic_pct_padding=.75, keep_n=8000)
train_body_vecs = body_pp.fit_transform(train_body_raw)

# process the title data
title_pp = processor(heuristic_pct_padding=.75, keep_n=4500)
train_title_vecs = title_pp.fit_transform(train_title_raw)

In [0]:
#Transform
test_body_raw = testdf.body.tolist()
test_title_raw = testdf.title.tolist()

test_body_vecs = body_pp.transform_parallel(test_body_raw)
test_title_vecs = title_pp.transform_parallel(test_title_raw)

In [0]:
train_labels = np.expand_dims(traindf.class_int.values, -1)
test_labels = np.expand_dims(testdf.class_int.values, -1)

In [0]:
import tensorflow as tf
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Embedding, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import Adam
import numpy as np

In [0]:
issue_body_doc_length = train_body_vecs.shape[1]
issue_title_doc_length = train_title_vecs.shape[1]

body_vocab_size = body_pp.n_tokens
title_vocab_size = title_pp.n_tokens

num_classes = len(set(train_labels[:, 0]))
assert num_classes == 3

In [0]:
#Building Model
body_input = Input(shape=(issue_body_doc_length,), name='Body-Input')
title_input = Input(shape=(issue_title_doc_length,), name='Title-Input')

body = Embedding(body_vocab_size, 50, name='Body-Embedding')(body_input)
title = Embedding(title_vocab_size, 50, name='Title-Embedding')(title_input)

body = BatchNormalization()(body)
body = GRU(100, name='Body-Encoder')(body)

title = BatchNormalization()(title)
title = GRU(75, name='Title-Encoder')(title)

x = Concatenate(name='Concat')([body, title])
x = BatchNormalization()(x)
out = Dense(num_classes, activation='softmax')(x)

model = Model([body_input, title_input], out)

model.compile(optimizer=Adam(lr=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
#Training
batch_size = 900
epochs = 4
history = model.fit(x=[train_body_vecs, train_title_vecs], 
                    y=train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=[(test_body_vecs, test_title_vecs), test_labels])


In [0]:
#Evaluation
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = np.argmax(best_model.predict(x=[test_body_vecs, test_title_vecs],
                                      batch_size=15000),
                   axis=1)

#Original Labels
y_test = test_labels[:, 0]

cm = confusion_matrix(y_test, 
                      y_pred)
print(cm)